# Ingestion del archivo language_rol.json

In [0]:
dbutils.widgets.text("p_environment", "")
v_env = dbutils.widgets.get("p_environment")
v_env

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Paso 1 - Leer el archivo CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StringType, StructField, IntegerType, DoubleType, StringType, DateType

In [0]:
language_role_schema = StructType(fields=[
    StructField("roleId", IntegerType(), True),
    StructField("languageRole", StringType(), True),
])

In [0]:
languages_roles_df = spark.read.json(f'{bronze_folder_path}/language_role.json', schema=language_role_schema, multiLine=True)

In [0]:
display(languages_roles_df)

In [0]:
languages_roles_df.printSchema()

### Paso 2 - Renombrar columnas y adicionar columnas "ingestion_date" y "env"

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp

In [0]:
languages_roles_renamed_df = (languages_roles_df
    .withColumnRenamed("roleId", "role_id")
    .withColumnRenamed("languageRole", "language_role")
    .withColumnRenamed("movieId", "movie_id")
    .transform(add_ingestion_date)
    .withColumn('env', lit(v_env))
)

### Paso 3 - Escribir datos en el DataLake en formato parquet

In [0]:
# languages_roles_renamed_df.write.mode('overwrite').parquet(f'{silver_folder_path}/language_role')
languages_roles_renamed_df.write.mode('overwrite').format('parquet').saveAsTable('movie_silver.languages_roles')

In [0]:
%python
display(dbutils.fs.ls("/mnt/moviehistory/silver/languages_roles"))

In [0]:
df = spark.read.parquet('/mnt/moviehistory/silver/languages_roles')
display(df)

In [0]:
dbutils.notebook.exit("Sucess")